# Variabel untuk diubah

In [ ]:
# Ukuran input model yang dikembangkan (jika menggunakan model saya, nilai ini jangan diubah)
PATCH_SIZE = (128, 128, 128)

# Jumlah patch yang diambil setiap batch
NUM_PATCH = 4

# Nilai overlap antara patch yang diambil
OVERLAP = 0.25

# Path ke model yang disimpan
MODEL_PATH = "Weights_Resize_v2_SGD\\best_metric_model.pth"

# Citra ToF input
INPUT_PATH = "D:\\ADAM Challenge Dataset\\Full Data\\10061F\\pre\\TOF.nii.gz"

# Lokasi penyimpanan hasil segmentasi
OUTPUT_PATH = "infer.nii"

# Infer

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import nibabel as nib
import torch

from monai.inferers import sliding_window_inference
from monai.transforms import (
	LoadImage,
    EnsureChannelFirst,
    Orientation,
    CropForeground,
    ScaleIntensityRange,
    AsDiscrete,
    Compose,
    EnsureType,
    Activations,
    Resize
)
from monai.networks.nets import UNETR
from monai.visualize import matshow3d

img_nib = nib.load(INPUT_PATH)
img_data = img_nib.get_fdata()
a_min = np.min(img_data)
a_max = np.max(img_data)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
	in_channels=1,
	out_channels=1,
	img_size=PATCH_SIZE,
	feature_size=16,
	hidden_size=768,
	mlp_dim=3072,
	num_heads=12,
	proj_type="perceptron",
	norm_name="instance",
	res_block=True,
	dropout_rate=0.0,
).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

post_label = AsDiscrete(threshold=0.5)
post_pred = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
to_numpy = Compose([EnsureType(data_type="numpy")])

infer_transforms = Compose(
    [
		LoadImage(image_only=True),
		EnsureChannelFirst(),
		Orientation(axcodes="RAS"),
		CropForeground(source_key=None),
		ScaleIntensityRange(a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0),
        Resize(spatial_size=(256,256,256))
	]
)

image = infer_transforms(INPUT_PATH)
image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).to(device)

with torch.no_grad():
    output = sliding_window_inference(image_tensor, roi_size=PATCH_SIZE, sw_batch_size=NUM_PATCH, overlap=OVERLAP, predictor=model, progress=True)
    output = post_pred(output)
    output_np = to_numpy(output[0]).squeeze(0)

nib_output = nib.Nifti1Image(output_np, affine=img_nib.affine)
nib.save(nib_output, OUTPUT_PATH)

# View Image

## Citra ToF

In [ ]:
# Citra ToF
fig = matshow3d(Orientation("SPL")(image), every_n=1)

In [ ]:
# Hasil segmentasi aneurisma
data1 = LoadImage(ensure_channel_first=True, simple_keys=True)(OUTPUT_PATH)
fig = matshow3d(Orientation("SPL")(data1), every_n=1)